<a href="https://colab.research.google.com/github/hieunguyen-cyber/AI_Project_CBH_GHHKM/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Huấn luyện mô hình nhận diện biển báo giao thông sử dụng YOLOv10**



Nhóm thực hiện: CBH_GHHKM

**Lưu ý**: Trước khi thực hiện, chuyển đổi môi trường chạy sang GPU

---

###**Bước 1**: Clone các file cần thiết

Ở đây, YOLOv10 sẽ được clone từ GitHub và tập dữ liệu sử dụng để huấn luyện sẽ được tải về sử dụng API từ Kaggle.

Nguồn:
* [Vietnamese Traffic Signs Dataset](https://www.kaggle.com/datasets/maitam/vietnamese-traffic-signs)
* [YOLOv10](https://github.com/THU-MIG/yolov10)

In [ ]:
!git clone https://github.com/hieunguyen-cyber/API-key.git
!git clone https://github.com/THU-MIG/yolov10

###**Bước 2**: Tiến hành tải về tập dữ liệu

Ở đây tôi đã thêm 1 file **sign.yaml** vào tập dữ liệu gốc nhằm phục vụ cho việc huấn luyện.

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/API-key/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle
!kaggle datasets download -d hieunguyen2208/vietnamese-traffic-signs

Tiến hành giải nén tập dữ liệu

In [3]:
!unzip /content/vietnamese-traffic-signs.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: dataset/images/1434.jpg  
  inflating: dataset/images/1435.jpg  
  inflating: dataset/images/1436.jpg  
  inflating: dataset/images/1437.jpg  
  inflating: dataset/images/1438.jpg  
  inflating: dataset/images/1439.jpg  
  inflating: dataset/images/1440.jpg  
  inflating: dataset/images/1441.jpg  
  inflating: dataset/images/1442.jpg  
  inflating: dataset/images/1443.jpg  
  inflating: dataset/images/1444.jpg  
  inflating: dataset/images/1445.jpg  
  inflating: dataset/images/1446.jpg  
  inflating: dataset/images/1447.jpg  
  inflating: dataset/images/1448.jpg  
  inflating: dataset/images/1449.jpg  
  inflating: dataset/images/1450.jpg  
  inflating: dataset/images/1451.jpg  
  inflating: dataset/images/1452.jpg  
  inflating: dataset/images/1453.jpg  
  inflating: dataset/images/1454.jpg  
  inflating: dataset/images/1455.jpg  
  inflating: dataset/images/1456.jpg  
  inflating: dataset/images/1457.jpg  
  infla

###**Bước 3**: Tiến hành cài đặt YOLOv10

**Lưu ý**: Do lỗi dependencies giữa một vài thư viện của PyTorch nên bước này chạy hai lần

In [ ]:
%cd yolov10
!pip install torch
!pip install -q -r requirements.txt
!pip install -e
!wget https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt

###**Bước 4**: Tiến hành chia tập dư liệu thành tập train và val

Cụ thể, tập dữ liệu gốc gồm 3216 cặp image-label, ta chia thành hai tập, tập train gồm 2572 cặp, tập val gồm 644 cặp

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

image_dir = '/content/dataset/images'
label_dir = '/content/dataset/labels'
train_image_dir = '/content/dataset/images/train'
val_image_dir = '/content/dataset/images/val'
train_label_dir = '/content/dataset/labels/train'
val_label_dir = '/content/dataset/labels/val'

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

if len(images) == 0:
    raise ValueError("No images found in the dataset directory.")
if len(images) < 5:
    raise ValueError("Not enough samples to split. Ensure you have at least 5 images.")

train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, train_size=0.8, random_state=42
)

for img, lbl in zip(train_images, train_labels):
    shutil.move(os.path.join(image_dir, img), os.path.join(train_image_dir, img))
    shutil.move(os.path.join(label_dir, lbl), os.path.join(train_label_dir, lbl))

for img, lbl in zip(val_images, val_labels):
    shutil.move(os.path.join(image_dir, img), os.path.join(val_image_dir, img))
    shutil.move(os.path.join(label_dir, lbl), os.path.join(val_label_dir, lbl))


###**Bước 5**: Tiến hành huấn luyện

Ở đây, do hạn chế về mặt phần cứng và để tiết kiệm thời gian huấn luyện, đội quyết định chọn các thông số như sau:
* EPOCHS = 30
* BATCH_SIZE = 6
* IMAGE_SIZE = 320

Cụ thể, với kích cỡ của tập dữ liệu như trên, thời gian huấn luyện 1 epoch trung bình mất 2 phút. Tổng thời gian huấn luyện khoảng 60 phút.

In [ ]:
from ultralytics import YOLOv10
MODEL_PATH = '/content/yolov10/yolov10n.pt'
model = YOLOv10(MODEL_PATH)
YAML_PATH = '/content/dataset/sign.yaml'
EPOCHS = 30
IMG_SIZE = 320
BATCH_SIZE = 3
model.train(data=YAML_PATH ,
            epochs= EPOCHS ,
            batch=BATCH_SIZE,
            imgsz= IMG_SIZE)

###**Bước 6**: Lưu file trọng số về máy

Tiến hành lưu file trọng số **best.pt** để tiến hành sử dụng trong bài toán nhận diện biển báo giao thông trong file **Prediction**

In [8]:
from google.colab import files

file_path = '/content/yolov10/runs/detect/train/weights/best.pt'
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>